[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pytosha/new-python-repo/blob/main/HomeTask/Task1_dmrf.ipynb)

# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание посложнее (опциональное, если очень хочется)
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [1]:
# pip install selenium
import requests
from tqdm.auto import tqdm
import time
import pandas as pd
from collections import Counter
from selenium import webdriver
from selenium.webdriver.common.by import By

## Шаг 1й - Получение списка объектов

In [2]:
# Для скачивания данных

# offset_ = 1 # размер шага
# limit_ = 11693 # количество объектов в одном ответе api
# url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
# res = requests.get(url)
# objects_data = res.json()
# objects_data.get('data').get('list')[0]

# Таким образом получить данные у меня не получилось, либо соединение прерывалось, либо бесконечно выполнялось (несколько часов)

In [3]:
# objects_list = objects_data.get('data').get('list')
# objids = [x.get('objId') for x in objects_list]
# objids[:10]
# objids

In [4]:
%%time
# ID всех объектов я собрал с помощью библиотеки Selenium (отрабатывает с учетом заданных задержек менее, чем за 20 минут)
all_ids = []
driver = webdriver.Chrome()
for k in tqdm(range(0, 118)):
    url = f"https://наш.дом.рф/сервисы/каталог-новостроек/список-объектов/список?page={k}&limit=100&objStatus=0"
    driver.get(url)
    driver.implicitly_wait(5)
    ids = driver.find_elements(By.CSS_SELECTOR, '.styles__Ellipsis-sc-1fw79ul-0.jYMONF.styles__Value-sc-1ajig0k-5.styles__IdValue-sc-1ajig0k-17.lodXzE.jTBlso');
    for i in ids:
        a = i.text
        b = a.replace('ID дома: ', "")
        if b != '':
            all_ids.append(b)

print(len(all_ids))
driver.close()

  0%|          | 0/118 [00:00<?, ?it/s]

11710
CPU times: total: 37.9 s
Wall time: 22min 31s


# Данные по определенному объекту

In [5]:
# Для скачивания данных по определенному объекту, нужно добавить его id в конец строки запроса
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/13397'

In [6]:
# Уберу дубли из полученного списка ID всех объектов
my_set = set(all_ids)
all_ids1 = list(my_set)

In [7]:
len(all_ids1)

11662

In [8]:
%%time
# соберу данные по каждому объекту
all_objects = []
for j in tqdm(range(len(all_ids1))):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{all_ids1[j]}'
    res = requests.get(url)
    object = res.json()
    # dict_one = {'id': vacancy['id'], 'published_at': vacancy['published_at'], 'name': vacancy['name'], 'description': vacancy['description'], 'key_skills': vacancy['key_skills']}
    all_objects.append(object['data'])
    if j !=0 and j % 10 == 0:
        time.sleep(1)

  0%|          | 0/11662 [00:00<?, ?it/s]

CPU times: total: 7min 40s
Wall time: 1h 42s


In [9]:
# добавлю данные в датафрейм
df=pd.DataFrame.from_dict(all_objects, orient='columns')

In [10]:
df.shape

(11662, 68)

In [11]:
df.isna().sum()

id                             0
hobjId                         0
pdId                           0
developer                      0
region                         0
                           ...  
metro                       9944
transportDistIndex          9511
transportDistIndexValue     9511
greenAreaIndexValue        10218
complexShortNm             10919
Length: 68, dtype: int64

In [12]:
# Cохранение pandas dataframe в excel
# pip install openpyxl
df.to_excel (r'C:\Users\PC\Desktop\new_buildings.xlsx')

In [13]:
# Cохранение pandas dataframe в pickle

In [14]:
df.to_pickle(r"C:\Users\PC\Desktop\new_buildings.pkl") 

In [15]:
# Cохранение pandas dataframe в sql 

In [16]:
# Чтобы сохранить датафрейм в SQL, изменим тип данных во всех столбцах на текстовый

In [17]:
df1 = df.astype(str)

In [18]:
df1.dtypes

id                         object
hobjId                     object
pdId                       object
developer                  object
region                     object
                            ...  
metro                      object
transportDistIndex         object
transportDistIndexValue    object
greenAreaIndexValue        object
complexShortNm             object
Length: 68, dtype: object

In [19]:
from sqlalchemy import create_engine

In [20]:
engine = create_engine(r'sqlite:///buildings.db', echo=True)
sqlite_connection = engine.connect()

In [21]:
sqlite_table = "new_buildings"
df1.to_sql(sqlite_table, sqlite_connection)

2023-12-18 13:00:47,439 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("new_buildings")
2023-12-18 13:00:47,440 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-12-18 13:00:47,443 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("new_buildings")
2023-12-18 13:00:47,444 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-12-18 13:00:47,460 INFO sqlalchemy.engine.Engine 
CREATE TABLE new_buildings (
	"index" BIGINT, 
	id TEXT, 
	"hobjId" TEXT, 
	"pdId" TEXT, 
	developer TEXT, 
	region TEXT, 
	address TEXT, 
	"nameObj" TEXT, 
	"floorMin" TEXT, 
	"floorMax" TEXT, 
	"objElemLivingCnt" TEXT, 
	"objReady100PercDt" TEXT, 
	"wallMaterialShortDesc" TEXT, 
	"objLkFinishTypeDesc" TEXT, 
	"objLkFreePlanDesc" TEXT, 
	"objElemParkingCnt" TEXT, 
	"objSquareLiving" TEXT, 
	"rpdNum" TEXT, 
	"rpdPdfLink" TEXT, 
	"rpdIssueDttm" TEXT, 
	"objLkLatitude" TEXT, 
	"objLkLongitude" TEXT, 
	"objCompensFundFlg" TEXT, 
	"objProblemFlg" TEXT, 
	"objLkClassDesc" TEXT, 
	"loadDttm" TEXT, 
	"photoRenderDTO" TEXT,

2023-12-18 13:00:50,073 INFO sqlalchemy.engine.Engine COMMIT


11662

In [22]:
sqlite_connection.close()